In [2]:
import numpy as np 
import pandas as pd 
from tqdm import tqdm 
import telegram
from transformers import AutoTokenizer, AutoModel
from transformers.optimization import get_linear_schedule_with_warmup, AdamW 
from sentence_transformers import SentenceTransformer, util, InputExample, losses, evaluation
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import pickle
import re
import os 
import time 
import datetime

In [ ]:
train_files = os.listdir("FGH_claim_triplet_v0.1s/train/")
valid_files = os.listdir("FGH_claim_triplet_v0.1s/valid/") 
test_files = os.listdir("FGH_claim_triplet_v0.1s/test/")

len(train_files), len(valid_files), len(test_files)

In [3]:
tel_token = "5059732158:AAE87TaReNbDKH3_Fy-CAYCUuIO2qiUyK2I"
chat_id = 1720119057 
bot = telegram.Bot(token=tel_token)

text = "Creating test triplets..." 
bot.sendMessage(chat_id=chat_id, text=text)

L = 30 

q_test, p_test, n_test = [], [], [] 

for i in tqdm(range(len(test_files)), position=0, leave=True): 
    try:
        if i % 100000 == 0:
            bot.sendMessage(chat_id=chat_id, text="currently at iteration {}/{}".format(i, len(test_files)))
    except Exception as e:
        print(e)
            
    file = open("FGH_claim_triplet_v0.1s/test/" + str(test_files[i]), "r") 
    claims = file.readlines() 
    
    query, positive, negative = [], [], []
    q, p, n = False, False, False 
    for j in range(len(claims)):
        if q == False:
            query.append(claims[j]) 
        elif p == False:
            positive.append(claims[j]) 
        else:
            negative.append(claims[j])
        if j < len(claims)-1 and claims[j] == '\n' and claims[j+1] == '\n':
            if q == False:
                q = True
            elif p == False:
                p = True 
    
    clean_query, clean_positive, clean_negative = [],[],[] 
    for q in query:
        if len(q.strip()) < L: 
            continue
        t = re.sub(r'\w[.)]\s*', '', q)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_query.append(t.strip()) 
    q_test.append(' '.join(clean_query)) 
    
    for p in positive:
        if len(p.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', p)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_positive.append(t.strip()) 
    p_test.append(' '.join(clean_positive))
    
    for n in negative:
        if len(n.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', n)
        t = re.sub('[^A-Za-z]+',' ',t)
        clean_negative.append(t.strip()) 
    n_test.append(' '.join(clean_negative))

100%|██████████| 50000/50000 [00:52<00:00, 948.41it/s] 


In [4]:
candidates = p_test + n_test
len(candidates)

100000

In [5]:
candidate_embeddings = [] 

model = SentenceTransformer("AI-Growth-Lab/PatentSBERTa")
for i in tqdm(range(len(candidates)), position=0, leave=True):
    doc = candidates[i] 
    embeddings = model.encode(doc) 
    candidate_embeddings.append(embeddings)

100%|██████████| 100000/100000 [1:39:22<00:00, 16.77it/s] 


In [6]:
import pickle 

bot.sendMessage(chat_id=chat_id, text = "saving candidate embeddings...")

with open("candidate_embs", "wb") as fp:   #Pickling
    pickle.dump(candidate_embeddings, fp)

bot.sendMessage(chat_id=chat_id, text = "Finished saving candidate embeddings!")
    


In [28]:
final_ranks = [] 
for idx in range(0,100): 
    q = q_test[idx]
    q_embedding = model.encode(q) 
    ranks = [] 
    for i in tqdm(range(len(candidate_embeddings)),position=0, leave=True):
        dist = util.cos_sim(q_embedding, candidate_embeddings[i]) 
        ranks.append((i, dist)) 

    ranks = sorted(ranks, key=lambda x: x[1], reverse=True)
    for i in range(len(ranks)):
        if ranks[i][0] == idx:
            print("rank = {}, score = {}".format(i+1, ranks[i][1].item())) 
            final_ranks.append(i+1) 
            break 

100%|██████████| 100000/100000 [00:18<00:00, 5515.92it/s]


rank = 510, score = 0.6412160396575928


100%|██████████| 100000/100000 [00:13<00:00, 7321.31it/s]


rank = 2576, score = 0.5892049074172974


100%|██████████| 100000/100000 [00:13<00:00, 7381.27it/s]


...

rank = 180, score = 0.6910422444343567


100%|██████████| 100000/100000 [00:12<00:00, 7831.04it/s]


rank = 28, score = 0.7023779153823853


100%|██████████| 100000/100000 [00:12<00:00, 7717.51it/s]


rank = 11864, score = 0.6401823163032532


In [34]:
mrrs = [] 
mrrs_100 = [] 
for i in range(len(final_ranks)):
    mrrs.append(1/final_ranks[i])
    if final_ranks[i] > 100: 
        mrrs_100.append(0) 
    else:
        mrrs_100.append(1/final_ranks[i]) 

print("MRR = {}".format(np.mean(mrrs))) 
print("MRR@100 = {}".format(np.mean(mrrs_100)))

MRR = 0.03638164556753714
MRR@100 = 0.03543998057799863


# Finetuning then making inference

In [5]:
tel_token = "5059732158:AAE87TaReNbDKH3_Fy-CAYCUuIO2qiUyK2I"
chat_id = 1720119057 
bot = telegram.Bot(token=tel_token)

text = "Creating train triplets..." 
bot.sendMessage(chat_id=chat_id, text=text)

L = 30 

q_train, p_train, n_train = [], [], [] 

for i in tqdm(range(len(train_files)), position=0, leave=True): 
    try:
        if i % 100000 == 0:
            bot.sendMessage(chat_id=chat_id, text="currently at iteration {}/{}".format(i, len(train_files)))
    except Exception as e:
        print(e)
            
    file = open("FGH_claim_triplet_v0.1s/train/" + str(train_files[i]), "r") 
    claims = file.readlines() 
    
    query, positive, negative = [], [], []
    q, p, n = False, False, False 
    for j in range(len(claims)):
        if q == False:
            query.append(claims[j]) 
        elif p == False:
            positive.append(claims[j]) 
        else:
            negative.append(claims[j])
        if j < len(claims)-1 and claims[j] == '\n' and claims[j+1] == '\n':
            if q == False:
                q = True
            elif p == False:
                p = True 
    
    clean_query, clean_positive, clean_negative = [],[],[] 
    for q in query:
        if len(q.strip()) < L: 
            continue
        t = re.sub(r'\w[.)]\s*', '', q)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_query.append(t.strip()) 
    q_train.append(' '.join(clean_query)) 
    
    for p in positive:
        if len(p.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', p)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_positive.append(t.strip()) 
    p_train.append(' '.join(clean_positive))
    
    for n in negative:
        if len(n.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', n)
        t = re.sub('[^A-Za-z]+',' ',t)
        clean_negative.append(t.strip()) 
    n_train.append(' '.join(clean_negative))

100%|██████████| 900000/900000 [19:57<00:00, 751.44it/s]


In [6]:
#Define your train examples. You need more than just two examples...
train_data = [] 
for i in tqdm(range(len(n_train))): 
    train_data.append(InputExample(texts=[q_train[i], p_train[i]], label=1.0)) 
    train_data.append(InputExample(texts=[q_train[i], n_train[i]], label=0.0))

100%|██████████| 900000/900000 [00:10<00:00, 86768.29it/s] 


In [7]:
model = SentenceTransformer("AI-Growth-Lab/PatentSBERTa", device='cuda')

train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16) 

train_loss = losses.CosineSimilarityLoss(model) 


In [8]:
# create evaluation data 
tel_token = "5059732158:AAE87TaReNbDKH3_Fy-CAYCUuIO2qiUyK2I"
chat_id = 1720119057 
bot = telegram.Bot(token=tel_token)

text = "Creating validation triplets..." 
bot.sendMessage(chat_id=chat_id, text=text)

L = 30 

q_valid, p_valid, n_valid = [], [], [] 

for i in tqdm(range(len(valid_files)), position=0, leave=True): 
    try:
        if i % 10000 == 0:
            bot.sendMessage(chat_id=chat_id, text="currently at iteration {}/{}".format(i, len(valid_files)))
    except Exception as e:
        print(e)
            
    file = open("FGH_claim_triplet_v0.1s/valid/" + str(valid_files[i]), "r") 
    claims = file.readlines() 
    
    query, positive, negative = [], [], []
    q, p, n = False, False, False 
    for j in range(len(claims)):
        if q == False:
            query.append(claims[j]) 
        elif p == False:
            positive.append(claims[j]) 
        else:
            negative.append(claims[j])
        if j < len(claims)-1 and claims[j] == '\n' and claims[j+1] == '\n':
            if q == False:
                q = True
            elif p == False:
                p = True 
    
    clean_query, clean_positive, clean_negative = [],[],[] 
    for q in query:
        if len(q.strip()) < L: 
            continue
        t = re.sub(r'\w[.)]\s*', '', q)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_query.append(t.strip()) 
    q_valid.append(' '.join(clean_query)) 
    
    for p in positive:
        if len(p.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', p)
        t = re.sub('[^A-Za-z]+',' ',t) 
        clean_positive.append(t.strip()) 
    p_valid.append(' '.join(clean_positive))
    
    for n in negative:
        if len(n.strip()) < L:
            continue 
        t = re.sub(r'\w[.)]\s*', '', n)
        t = re.sub('[^A-Za-z]+',' ',t)
        clean_negative.append(t.strip()) 
    n_valid.append(' '.join(clean_negative))

100%|██████████| 50000/50000 [01:10<00:00, 709.39it/s]


In [9]:
sent1, sent2, scores = [], [], [] 
for i in tqdm(range(len(n_valid))):
    sent1.append(q_valid[i]) 
    sent2.append(p_valid[i]) 
    scores.append(1.0) 
    
    sent1.append(q_valid[i]) 
    sent2.append(n_valid[i]) 
    scores.append(0.0) # negative samples 

100%|██████████| 50000/50000 [00:00<00:00, 333023.47it/s]


In [10]:
evaluator = evaluation.EmbeddingSimilarityEvaluator(sent1, sent2, scores)

In [13]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=1, 
          warmup_steps=100, 
          evaluator=evaluator, 
          evaluation_steps=10000, 
          save_best_model=True, 
          output_path='PatentSBERTa_900000_finetuned', 
          show_progress_bar = True)  

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/112500 [00:00<?, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

In [14]:
bot.sendMessage(chat_id = chat_id, text = "Finished Training!")

In [15]:
for i in range(10):
    bot.sendMessage(chat_id = chat_id, text = "PatentSBERTa training Done!!!") 
    time.sleep(2) 

In [17]:
test_model = SentenceTransformer("PatentSBERTa_900000_finetuned", device='cuda') 
test_model 

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [18]:
candidate_embeddings = [] 

for i in tqdm(range(0, len(candidates), 8), position=0, leave=True):
    doc = candidates[i:i+8] 
    embeddings = test_model.encode(doc) 
    for j in range(8): 
        candidate_embeddings.append(embeddings[j, :])

100%|██████████| 12500/12500 [23:26<00:00,  8.89it/s]


In [20]:
print("saving tabnet_900000_finetuned_embeddings...") 
with open("tabnet_900000_finetuned.pkl", "wb") as f: 
    pickle.dump(candidate_embeddings, f) 

saving tabnet_900000_finetuned_embeddings...


In [27]:
final_ranks = [] 
final_reciprocal_ranks = [] 
for idx in range(0,len(q_test)): 
    if idx%100 == 0 and idx != 0:
        bot.sendMessage(chat_id=chat_id, text="Currently at iteration {}/{} for inference".format(idx, len(q_test)))
        cur_mean_rank = np.mean(final_ranks)  
        cur_mrr = np.mean(final_reciprocal_ranks)
        text = "current mean rank is {}".format(cur_mean_rank) 
        bot.sendMessage(chat_id=chat_id, text=text)  
        text = "current mrr is {}".format(cur_mrr) 
        bot.sendMessage(chat_id=chat_id, text=text)
        
    q = q_test[idx]
    q_embedding = test_model.encode(q) 
    ranks = [] 
    for i in tqdm(range(len(candidate_embeddings)),position=0, leave=True):
        dist = util.cos_sim(q_embedding, candidate_embeddings[i]) 
        ranks.append((i, dist)) 

    ranks = sorted(ranks, key=lambda x: x[1], reverse=True)
    for i in range(len(ranks)):
        if ranks[i][0] == idx:
            print("rank = {}, score = {}".format(i+1, ranks[i][1].item())) 
            final_ranks.append(i+1) 
            final_reciprocal_ranks.append(1/(i+1)) 
            break 

100%|██████████| 100000/100000 [00:13<00:00, 7218.23it/s]


rank = 1, score = 0.912592351436615


100%|██████████| 100000/100000 [00:13<00:00, 7238.67it/s]


rank = 3, score = 0.9978505969047546


100%|██████████| 100000/100000 [00:13<00:00, 7626.02it/s]


...

rank = 1, score = 0.9575493335723877


100%|██████████| 100000/100000 [00:13<00:00, 7174.86it/s]


rank = 7, score = 0.855083703994751


100%|██████████| 100000/100000 [00:13<00:00, 7221.03it/s]


KeyboardInterrupt: 

In [ ]:
for i in range(10): 
    bot.sendMessage(chat_id=chat_id, text="inference complete!") 
    time.sleep(2) 

In [ ]:
bot.sendMessage(chat_id=chat_id, text="saving final ranks array...") 
with open("tabnet_900000_test_set_final_ranks_arr.pkl", "wb") as f: 
    pickle.dump(final_ranks, f) 

In [30]:
mrr = [] 
mrr_100 = [] 
for i in range(len(final_ranks[:75])):
    mrr.append(1/final_ranks[i]) 
    if final_ranks[i] > 100:
        mrr_100.append(0)
    else:
        mrr_100.append(1/final_ranks[i])
        
print("mrr = {}".format(np.mean(mrr))) 
print("mrr@100 = {}".format(np.mean(mrr_100))) 

mrr = 0.05811983470946649
mrr@100 = 0.0567677216934171
